In [ ]:
import pickle
import torch.nn as nn
import torch
import pandas as pd
from torch.utils.data import DataLoader, Dataset, Subset
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.metrics import mean_absolute_error,r2_score, mean_squared_error

## 0. Dataset and DataLoader

In [ ]:
name='CAISO_zone_1_.csv'
surrogate_model_path = 'transformer_wind_power_model2_CAISO_zone_1_.pth'
with open('weather_scaler.pkl', 'rb') as f:
    weather_scaler = pickle.load(f)

class WindPowerDataset(Dataset):
    def __init__(self, csv_file, wind_power_scaler=None, weather_scaler=None, save_scalers=False):
        self.data = pd.read_csv(csv_file)

        self.data = self.data.iloc[::5, :].reset_index(drop=True)

        self.original_wind_power_std = self.data.iloc[:, 2].std()
        self.original_weather_std = self.data.iloc[:, 4:12].std()

        if wind_power_scaler is None or weather_scaler is None:
            self.wind_power_scaler = MinMaxScaler()
            self.weather_scaler = MinMaxScaler()

            self.data.iloc[:, 2] = self.wind_power_scaler.fit_transform(self.data.iloc[:, 2].values.reshape(-1, 1)).squeeze()

            self.data.iloc[:, 4:12] = self.weather_scaler.fit_transform(self.data.iloc[:, 4:12])
            
            if save_scalers:
                with open('wind_power_scaler.pkl', 'wb') as f:
                    pickle.dump(self.wind_power_scaler, f)
                with open('weather_scaler.pkl', 'wb') as f:
                    pickle.dump(self.weather_scaler, f)
        else:
            self.wind_power_scaler = wind_power_scaler
            self.weather_scaler = weather_scaler
            self.data.iloc[:, 2] = self.wind_power_scaler.transform(self.data.iloc[:, 2].values.reshape(-1, 1)).squeeze()
            self.data.iloc[:, 4:12] = self.weather_scaler.transform(self.data.iloc[:, 4:12])
    
    def __len__(self):
        return len(self.data) - 312  # 288 (1440/5) + 24 (120/5)
    
    def __getitem__(self, idx):
        wind_power_history = self.data.iloc[idx:idx + 288, 2].values.astype(float)
        future_weather = self.data.iloc[idx + 288:idx + 312, 4:12].values.astype(float)
        future_wind_power = self.data.iloc[idx + 312, 2]
        return torch.tensor(wind_power_history, dtype=torch.float32), \
               torch.tensor(future_weather, dtype=torch.float32), \
               torch.tensor(future_wind_power, dtype=torch.float32)
    
    def get_original_stds(self):
        return {
            'original_wind_power_std': self.original_wind_power_std,
            'original_weather_std': self.original_weather_std.to_dict()
        }

dataset = WindPowerDataset(name, save_scalers=True)
weather_stds = dataset.get_original_stds()
weather_stds_array = np.array(list(weather_stds['original_weather_std'].values()))  
weather_data = pd.read_csv(name)
weather_data = weather_data.iloc[::5, :].reset_index(drop=True)
weather_mean = weather_data.iloc[:, 4:12].mean()
wind_power_scaler=dataset.wind_power_scaler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size=32
test_split=0.2
test_size = int(len(dataset) * test_split)
train_size = len(dataset) - test_size
train_dataset = Subset(dataset, list(range(train_size)))
test_dataset = Subset(dataset, list(range(train_size, len(dataset))))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## 1. Load surrogate model and target model

### traget model

In [ ]:
class WindPowerPredictor(nn.Module):
    def __init__(self):
        super(WindPowerPredictor, self).__init__()
        self.lstm_wind = nn.LSTM(input_size=1, hidden_size=128, num_layers=1, batch_first=True)
        self.lstm_weather = nn.LSTM(input_size=8, hidden_size=128, num_layers=1, batch_first=True)
        self.fc = nn.Linear(256, 1)
        self.relu=nn.ReLU()
    
    def forward(self, wind_history, weather_future):
        wind_history = wind_history.unsqueeze(-1)
        _, (hn_wind, _) = self.lstm_wind(wind_history)
        _, (hn_weather, _) = self.lstm_weather(weather_future)
        hn_wind = hn_wind[-1, :, :]
        hn_weather = hn_weather[-1, :, :]
        combined = torch.cat((hn_wind, hn_weather), dim=1)
        output = self.fc(combined)
        output = self.relu(output)
        return output

In [ ]:
def load_model(model_path, device='cpu'):
    model = WindPowerPredictor().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = 'wind_caiso_lstm_sigmoid_version2.pth'
model = load_model(model_path, device)

### surrogate model

In [ ]:
class WindPowerPredictor(nn.Module):  
    def __init__(self):  
        super(WindPowerPredictor, self).__init__()  
        self.gru_wind = nn.GRU(input_size=1, hidden_size=50, num_layers=2, batch_first=True)  
        self.gru_weather = nn.GRU(input_size=8, hidden_size=50, num_layers=2, batch_first=True)  
        self.fc = nn.Linear(100, 1)  
        self.sigmoid = nn.Sigmoid()
      
    def forward(self, wind_history, weather_future):  
        wind_history = wind_history.unsqueeze(-1)  
        _, hn_wind = self.gru_wind(wind_history)  
        _, hn_weather = self.gru_weather(weather_future)  
        hn_wind = hn_wind[-1, :, :]  
        hn_weather = hn_weather[-1, :, :]  
        combined = torch.cat((hn_wind, hn_weather), dim=1)  
        output = self.fc(combined)  
        output = self.sigmoid(output)
        return output  
    
model_path1 = 'wind_gru_caiso_sigmoid.pth'
model1 = load_model(model_path1, device)

In [ ]:
class WindPowerPredictor(nn.Module):
    def __init__(self, d_model=50):
        super(WindPowerPredictor, self).__init__()
        self.d_model = d_model
        self.embedding_wind = nn.Linear(1, d_model)
        self.embedding_weather = nn.Linear(8, d_model)
        self.transformer_wind = nn.Transformer(
            d_model=d_model, nhead=2, num_encoder_layers=2, num_decoder_layers=2, dim_feedforward=200, dropout=0.1
        )
        self.transformer_weather = nn.Transformer(
            d_model=d_model, nhead=2, num_encoder_layers=2, num_decoder_layers=2, dim_feedforward=200, dropout=0.1
        )
        self.fc = nn.Linear(d_model * 2, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, wind_history, weather_future):
        wind_history = self.embedding_wind(wind_history.unsqueeze(-1))  # (batch_size, seq_len, d_model)
        wind_history = wind_history.permute(1, 0, 2)  # (seq_len, batch_size, d_model)
        weather_future = self.embedding_weather(weather_future)  # (batch_size, seq_len, d_model)
        weather_future = weather_future.permute(1, 0, 2)  # (seq_len, batch_size, d_model)

        transformer_output_wind = self.transformer_wind(wind_history, wind_history)
        transformer_output_weather = self.transformer_weather(weather_future, weather_future)
        
        combined = torch.cat((transformer_output_wind[-1, :, :], transformer_output_weather[-1, :, :]), dim=1)
        output = self.fc(combined)
        output = self.sigmoid(output)  
        return output
    
model_path2 = 'wind_caiso_transformer_sigmoid.pth'
model2 = load_model(model_path2, device)

In [ ]:
class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=(kernel_size-1) * dilation_size, dilation=dilation_size),
                       nn.ReLU(),
                       nn.Dropout(dropout)]
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

class WindPowerPredictorTCN(nn.Module):
    def __init__(self):
        super(WindPowerPredictorTCN, self).__init__()
        self.tcn_wind = TemporalConvNet(num_inputs=1, num_channels=[50]*3, kernel_size=3, dropout=0.2)
        self.tcn_weather = TemporalConvNet(num_inputs=8, num_channels=[50]*3, kernel_size=3, dropout=0.2)
        self.fc = nn.Linear(50 * 2, 1)  # Combined output size of TCNs
        self.sigmoid = nn.Sigmoid()

    def forward(self, wind_history, weather_future):
        wind_history = wind_history.unsqueeze(1)  # (batch_size, 1, seq_len)
        tcn_output_wind = self.tcn_wind(wind_history).transpose(1, 2)[:, -1, :]
        
        weather_future = weather_future.transpose(1, 2)  # (batch_size, 8, seq_len)
        tcn_output_weather = self.tcn_weather(weather_future).transpose(1, 2)[:, -1, :]
        
        combined = torch.cat((tcn_output_wind, tcn_output_weather), dim=1)
        output = self.fc(combined)
        output = self.sigmoid(output)
        return output
    
model3 = WindPowerPredictorTCN()
model_path3 = 'wind_tcn_caiso_sigmoid.pth'
model3.load_state_dict(torch.load(model_path3)).to(device)

In [ ]:
class WindPowerPredictor(nn.Module):
    def __init__(self):
        super(WindPowerPredictor, self).__init__()
        self.lstm_wind = nn.LSTM(input_size=1, hidden_size=50, num_layers=2, batch_first=True)
        self.lstm_weather = nn.LSTM(input_size=8, hidden_size=50, num_layers=2, batch_first=True)
        self.fc = nn.Linear(100, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, wind_history, weather_future):
        wind_history = wind_history.unsqueeze(-1)
        _, (hn_wind, _) = self.lstm_wind(wind_history)
        _, (hn_weather, _) = self.lstm_weather(weather_future)
        hn_wind = hn_wind[-1, :, :]
        hn_weather = hn_weather[-1, :, :]
        combined = torch.cat((hn_wind, hn_weather), dim=1)
        output = self.fc(combined)
        output = self.sigmoid(output)  
        return output
    
model_path4 = 'wind_caiso_lstm_sigmoid.pth'
model4 = WindPowerPredictor()
model4.load_state_dict(torch.load(model_path4)).to(device)

## Attack peroformance

### RA

In [ ]:
def adversarial_attack_RA(model, data_loader, epsilon=0.01, device='cpu'):
    model.eval()  
    all_preds = []
    all_adversarial_preds = []
    all_targets = []
    all_original_weather = []
    all_adversarial_weather = []
    
    for parameters in model.parameters():
        parameters.requires_grad = False
    
    for wind_history, weather_future, future_wind_power in data_loader:
        wind_history = wind_history.to(device)
        weather_future = weather_future.to(device)
        future_wind_power = future_wind_power.to(device)
        original_output = model(wind_history, weather_future)
        uap=torch.empty_like(weather_future).uniform_(-epsilon, epsilon)
        adversarial_weather = weather_future +uap
        adversarial_weather = torch.clamp(adversarial_weather, 0, 1)  
        adversarial_output = model(wind_history, adversarial_weather.detach())  
        all_preds.append(original_output.detach().cpu().numpy())
        all_adversarial_preds.append(adversarial_output.detach().cpu().numpy())
        all_targets.append(future_wind_power.cpu().numpy())
        all_original_weather.append(weather_future.detach().cpu().numpy())
        all_adversarial_weather.append(adversarial_weather.detach().cpu().numpy())

    model.eval()  

    return all_preds, all_adversarial_preds, all_targets, all_original_weather, all_adversarial_weather

In [ ]:
epsilon_list=[0.03,0.05,0.1]
for epsilon in epsilon_list:
    original_preds, adversarial_preds, targets, original_weathers, adversarial_weathers = adversarial_attack_RA(model, test_loader, epsilon=epsilon, device=device)
    all_targets = np.concatenate(targets).flatten()
    all_adversarial_preds=np.concatenate(adversarial_preds).flatten()
    all_targets_inv=wind_power_scaler.inverse_transform(np.array(all_targets).reshape(-1, 1)).squeeze()
    all_preds_inv=wind_power_scaler.inverse_transform(np.array(all_adversarial_preds).reshape(-1, 1)).squeeze()
    rmse = mean_squared_error(all_targets_inv, all_preds_inv, squared=False)
    mse = mean_squared_error(all_targets_inv, all_preds_inv)
    print(f'RMSE_inv: {rmse:.10f}')
    print(f'MSE_inv: {mse:.10f}')
    mae = mean_absolute_error(all_targets_inv, all_preds_inv)
    print(f'MAE_inv: {mae:.10f}')
    print('---------------------------------------------------')

### FGSM

In [ ]:
def adversarial_attack_FGSM(sur_model, model, data_loader, epsilon=0.01, device='cpu'):
    sur_model.train()  
    model.eval()
    all_preds = []
    all_adversarial_preds = []
    all_targets = []
    all_original_weather = []
    all_adversarial_weather = []

    for parameters in sur_model.parameters():
        parameters.requires_grad = False
    
    for wind_history, weather_future, future_wind_power in data_loader:
        wind_history = wind_history.to(device)
        weather_future = weather_future.to(device)
        future_wind_power = future_wind_power.to(device)
        weather_future.requires_grad = True
        original_output = model(wind_history, weather_future)
        sur_original_output = sur_model(wind_history, weather_future)
        original_loss = torch.sum(sur_original_output.squeeze()- future_wind_power)
        sur_model.zero_grad()
        original_loss.backward()  

        # adversarial perturbation
        weather_future_grad = weather_future.grad.data
        adversarial_weather = weather_future + epsilon * weather_future_grad.sign()
        adversarial_weather = torch.clamp(adversarial_weather, 0, 1)  # ensure the data is within a reasonable range
        adversarial_output = model(wind_history, adversarial_weather.detach())  

        all_preds.append(original_output.detach().cpu().numpy())
        all_adversarial_preds.append(adversarial_output.detach().cpu().numpy())
        all_targets.append(future_wind_power.cpu().numpy())
        all_original_weather.append(weather_future.detach().cpu().numpy())
        all_adversarial_weather.append(adversarial_weather.detach().cpu().numpy())

    model.eval()  

    return all_preds, all_adversarial_preds, all_targets, all_original_weather, all_adversarial_weather

In [ ]:
epsilon_list=[0.03,0.05,0.1]
sur_model_list=[model1,model2,model3,model4]
for sur_model in sur_model_list:
    for epsilon in epsilon_list:
        original_preds, adversarial_preds, targets, original_weathers, adversarial_weathers = adversarial_attack_FGSM(sur_model, model, test_loader, epsilon=epsilon, device=device)
        all_targets = np.concatenate(targets).flatten()
        all_adversarial_preds=np.concatenate(adversarial_preds).flatten()
        all_targets_inv=wind_power_scaler.inverse_transform(np.array(all_targets).reshape(-1, 1)).squeeze()
        all_preds_inv=wind_power_scaler.inverse_transform(np.array(all_adversarial_preds).reshape(-1, 1)).squeeze()
        rmse = mean_squared_error(all_targets_inv, all_preds_inv, squared=False)
        mse = mean_squared_error(all_targets_inv, all_preds_inv)
        print(f'RMSE_inv: {rmse:.10f}')
        print(f'MSE_inv: {mse:.10f}')
        mae = mean_absolute_error(all_targets_inv, all_preds_inv)
        print(f'MAE_inv: {mae:.10f}')
        print('---------------------------------------------------')

### PGD

In [ ]:
def adversarial_attack_PGD(sur_model, model, data_loader, epsilon=0.01, alpha=0.003, num_iterations=80, device='cpu'):
    sur_model.train()  
    model.eval()
    all_preds = []
    all_adversarial_preds = []
    all_targets = []
    all_original_weather = []
    all_adversarial_weather = []

    for parameters in sur_model.parameters():
        parameters.requires_grad = False

    for wind_history, weather_future, future_wind_power in data_loader:
        wind_history = wind_history.to(device)
        weather_future = weather_future.to(device)
        future_wind_power = future_wind_power.to(device)
        adversarial_weather = weather_future.clone().detach()
        adversarial_weather.requires_grad = True
        original_output = model(wind_history, weather_future)

        sur_original_output = sur_model(wind_history, weather_future)
        for i in range(num_iterations):
            original_output = model(wind_history, adversarial_weather)
            
            sur_original_output = sur_model(wind_history, adversarial_weather)
            original_loss = torch.sum(sur_original_output.squeeze()- future_wind_power)
            sur_model.zero_grad()
            original_loss.backward()  
            with torch.no_grad():
                weather_future_grad = adversarial_weather.grad.data
                adversarial_weather += alpha * weather_future_grad.sign()
                adversarial_weather = torch.max(torch.min(adversarial_weather, weather_future + epsilon),
                                                weather_future - epsilon)
                adversarial_weather = torch.clamp(adversarial_weather, 0,1)
                adversarial_weather = adversarial_weather.detach().requires_grad_(True)
     
        adversarial_output = model(wind_history, adversarial_weather)
        original_output = model(wind_history, weather_future)

        all_preds.append(original_output.detach().cpu().numpy())
        all_adversarial_preds.append(adversarial_output.detach().cpu().numpy())
        all_targets.append(future_wind_power.cpu().numpy())
        all_original_weather.append(weather_future.detach().cpu().numpy())
        all_adversarial_weather.append(adversarial_weather.detach().cpu().numpy())
    sur_model.eval()  
    
    return all_preds, all_adversarial_preds, all_targets, all_original_weather, all_adversarial_weather

In [ ]:
epsilon_list=[0.03,0.05,0.1]
sur_model_list=[model1,model2,model3,model4]
for sur_model in sur_model_list:
    for epsilon in epsilon_list:
        original_preds, adversarial_preds, targets, original_weathers, adversarial_weathers = adversarial_attack_PGD(sur_model, model, test_loader, epsilon=epsilon, device=device)
        all_targets = np.concatenate(targets).flatten()
        all_adversarial_preds=np.concatenate(adversarial_preds).flatten()
        all_targets_inv=wind_power_scaler.inverse_transform(np.array(all_targets).reshape(-1, 1)).squeeze()
        all_preds_inv=wind_power_scaler.inverse_transform(np.array(all_adversarial_preds).reshape(-1, 1)).squeeze()
        rmse = mean_squared_error(all_targets_inv, all_preds_inv, squared=False)
        mse = mean_squared_error(all_targets_inv, all_preds_inv)
        print(f'RMSE_inv: {rmse:.10f}')
        print(f'MSE_inv: {mse:.10f}')
        mae = mean_absolute_error(all_targets_inv, all_preds_inv)
        print(f'MAE_inv: {mae:.10f}')
        print('---------------------------------------------------')

### AoA

In [ ]:
from captum.attr import IntegratedGradients
import torch.optim as optim

def compute_attention(model, wind_history, weather_input, baseline_type='zero'):
    was_training = model.training
    try:
        model.train()
        if not weather_input.requires_grad:
            weather_input.requires_grad_(True)

        with torch.backends.cudnn.flags(enabled=False):
            output = model(wind_history, weather_input)

        grad_outputs = torch.ones_like(output)
        
        grads = torch.autograd.grad(
            outputs=output,
            inputs=weather_input,
            grad_outputs=grad_outputs,
            create_graph=True
        )[0]

        attention = torch.abs(grads * weather_input)
        return attention.contiguous()

    finally:
        if not was_training:
            model.eval()

def generate_universal_adversarial_samples(
    surrogate_models,
    test_loader,
    device,
    lambda_aoa=1.0,
    lambda_reg=10.0,
    epsilon=0.1,  
    num_steps=10,
    learning_rate=0.01,
    target_bias=0.1, 
    save_path="universal_adversarial_samples.pkl"
):
   
    for model in surrogate_models:
        model.train() 

    adversarial_samples = []

    for batch_idx, (wind_history, future_weather, true_power) in enumerate(test_loader):
        wind_history = wind_history.to(device)
        future_weather = future_weather.to(device)
        true_power = true_power.to(device)

        A_ori_list = []
        for model in surrogate_models:
            A_ori = compute_attention(model, wind_history, future_weather, baseline_type='original')
            A_ori_list.append(A_ori.detach())

        future_weather_adv = future_weather.detach().clone().requires_grad_(True)
        optimizer = optim.Adam([future_weather_adv], lr=learning_rate)

        for step in range(num_steps):
            optimizer.zero_grad()
            total_loss = 0.0

            for i, model in enumerate(surrogate_models):
                A_adv = compute_attention(model, wind_history, future_weather_adv, baseline_type='original')
                A_ori_flat = A_ori_list[i].reshape(A_ori_list[i].size(0), -1)  
                A_adv_flat = A_adv.reshape(A_adv.size(0), -1)
                cos_sim = nn.functional.cosine_similarity(A_ori_flat, A_adv_flat, dim=1)
                L_aoa = -cos_sim.mean()

                pred_adv = model(wind_history, future_weather_adv).squeeze()
                pred_ori = model(wind_history, future_weather).squeeze().detach()
                L_reg = -pred_adv.mean()

                loss = lambda_aoa * L_aoa + lambda_reg * L_reg
                total_loss += loss

            total_loss.backward()
            optimizer.step()

            with torch.no_grad():
                future_weather_adv.data = torch.clamp(
                    future_weather_adv,
                    min=future_weather - epsilon,
                    max=future_weather + epsilon
                )

        adversarial_samples.append({
            'index': batch_idx,
            'wind_history': wind_history.cpu().detach().numpy(),
            'future_weather_original': future_weather.cpu().detach().numpy(),
            'future_weather_adversarial': future_weather_adv.cpu().detach().numpy(),
            'true_power': true_power.cpu().detach().numpy()
        })


        if batch_idx % 100 == 0:
            print(f"Batch {batch_idx}, Loss: {total_loss.item():.4f}")
    return adversarial_samples

In [ ]:
def evaluate_attack_with_mae(surrogate_models, adversarial_samples, device, wind_power_scaler=None):
    results = []
    for model_idx, model in enumerate(surrogate_models):
        model.eval()
        total_bias = 0.0
        total_mae_ori = 0.0
        total_mae_adv = 0.0
        count = 0
        
        for sample in adversarial_samples:
            wind_history = torch.tensor(sample['wind_history'], dtype=torch.float32).to(device)
            future_weather_ori = torch.tensor(sample['future_weather_original'], dtype=torch.float32).to(device)
            future_weather_adv = torch.tensor(sample['future_weather_adversarial'], dtype=torch.float32).to(device)
            true_power = torch.tensor(sample['true_power'], dtype=torch.float32).to(device)
            
            with torch.no_grad():
                pred_ori = model(wind_history, future_weather_ori).squeeze()
                pred_adv = model(wind_history, future_weather_adv).squeeze()
                
                bias = (pred_adv - pred_ori).mean().item()
                total_bias += bias
                
                mae_ori = torch.abs(pred_ori - true_power).mean().item()
                mae_adv = torch.abs(pred_adv - true_power).mean().item()
                total_mae_ori += mae_ori
                total_mae_adv += mae_adv
                
                count += 1
        
        avg_bias = total_bias / count
        avg_mae_ori = total_mae_ori / count
        avg_mae_adv = total_mae_adv / count
        mae_increase = avg_mae_adv - avg_mae_ori
        
        result = {
            'model_idx': model_idx,
            'avg_prediction_bias': avg_bias,
            'avg_mae_original': avg_mae_ori,
            'avg_mae_adversarial': avg_mae_adv,
            'mae_increase': mae_increase
        }
        results.append(result)
        
        print(f"Model {model_idx}:")
        print(f"adversarial MAE = {avg_mae_adv:.4f}")
    return results

In [ ]:
def run_aoa_attack_and_save(
    surrogate_models,
    test_loader,
    device,
    wind_power_scaler,
    epsilon=0.1,
    num_steps=20,
    learning_rate=0.01,
    lambda_aoa=1.0,
    lambda_reg=10.0,
    model_idx_to_use=0,  
    save_csv_name="buchong_AoA.csv"
):

    print(f"\n AoA attack (epsilon={epsilon})...")

    adversarial_samples = generate_universal_adversarial_samples(
        surrogate_models=surrogate_models,
        test_loader=test_loader,
        device=device,
        lambda_aoa=lambda_aoa,
        lambda_reg=lambda_reg,
        epsilon=epsilon,
        num_steps=num_steps,
        learning_rate=learning_rate,
        target_bias=0.0,  
        save_path=f"adversarial_samples_epsilon_{epsilon}.pkl"
    )

    print(f"\n evaluate attack performance (epsilon={epsilon})...")
    attack_results = evaluate_attack_with_mae(
        surrogate_models,
        adversarial_samples,
        device,
        wind_power_scaler=wind_power_scaler
    )


    model = surrogate_models[model_idx_to_use]
    model.eval()

    all_targets = []      
    all_preds_adv = []    

    for sample in adversarial_samples:
        wind_history = torch.tensor(sample['wind_history'], dtype=torch.float32).to(device)
        future_weather_adv = torch.tensor(sample['future_weather_adversarial'], dtype=torch.float32).to(device)
        true_power = torch.tensor(sample['true_power'], dtype=torch.float32).to(device)

        with torch.no_grad():
            pred_adv = model(wind_history, future_weather_adv).squeeze()

        all_targets.append(true_power.cpu().numpy())
        all_preds_adv.append(pred_adv.cpu().numpy())

    all_targets = np.concatenate(all_targets).flatten()
    all_preds_adv = np.concatenate(all_preds_adv).flatten()

    r2_adv = r2_score(all_targets, all_preds_adv)
    rmse_adv = mean_squared_error(all_targets, all_preds_adv, squared=False)
    mse_adv = mean_squared_error(all_targets, all_preds_adv)
    print(f'R²: {r2_adv:.10f}')
    print(f'RMSE: {rmse_adv:.10f}')
    print(f'MSE: {mse_adv:.10f}')

    return attack_results, r2_adv, rmse_adv, mse_adv

In [ ]:
epsilons = [0.1, 0.05, 0.03]
csv_names = ['buchong_AoA_010.csv', 'buchong_AoA_005.csv', 'buchong_AoA_003.csv']

for eps, csv_name in zip(epsilons, csv_names):
    print(f"\n{'='*50}")
    print(f" AoA attack, epsilon={eps}")
    print(f"{'='*50}")

    attack_results, r2, rmse, mse = run_aoa_attack_and_save(
        surrogate_models=sur_model_list,
        test_loader=test_loader,
        device=device,
        wind_power_scaler=wind_power_scaler,
        epsilon=eps,
        num_steps=80,        
        learning_rate=0.003,  
        lambda_aoa=1.0,
        lambda_reg=10.0,
        model_idx_to_use=0,  
        save_csv_name=csv_name
    )

    for res in attack_results:
        if res['model_idx'] == 0:
            print(f" Model 0 - MAE Increase: {res['mae_increase']:.6f}")
            break

## UP

In [ ]:
def adversarial_attack_inde(uap_loaded, model, data_loader, device='cpu'):
    model.eval()  
    all_preds = []
    all_adversarial_preds = []
    all_targets = []
    all_original_weather = []
    all_adversarial_weather = []
    for wind_history, weather_future, future_wind_power in data_loader:
        wind_history = wind_history.to(device)
        weather_future = weather_future.to(device)
        future_wind_power = future_wind_power.to(device)
        original_output = model(wind_history, weather_future)
        adversarial_weather = weather_future + uap_loaded
        adversarial_weather = torch.clamp(adversarial_weather, 0, 1)  
        adversarial_output = model(wind_history, adversarial_weather.detach())  
        all_preds.append(original_output.detach().cpu().numpy())
        all_adversarial_preds.append(adversarial_output.detach().cpu().numpy())
        all_targets.append(future_wind_power.cpu().numpy())
        all_original_weather.append(weather_future.detach().cpu().numpy())
        all_adversarial_weather.append(adversarial_weather.detach().cpu().numpy())
    model.eval()  
    return all_preds, all_adversarial_preds, all_targets, all_original_weather, all_adversarial_weather

In [ ]:
with open('uap_list010_caiso.pkl', 'rb') as f:
    uap_list010 = pickle.load(f)

with open('uap_list005_caiso.pkl', 'rb') as f:
    uap_list005 = pickle.load(f)

with open('uap_list003_caiso.pkl', 'rb') as f:
    uap_list003 = pickle.load(f)

In [ ]:
for uap_loaded in uap_list010:
    original_preds, adversarial_preds, targets, original_weathers, adversarial_weathers = adversarial_attack_inde(uap_loaded, model, test_loader, device=device)
    adversarial_preds_uni = np.concatenate(adversarial_preds).flatten()
    targets= np.concatenate(targets).flatten()
    all_targets_inv=wind_power_scaler.inverse_transform(np.array(targets).reshape(-1, 1)).squeeze()
    all_preds_inv=wind_power_scaler.inverse_transform(np.array(adversarial_preds_uni).reshape(-1, 1)).squeeze()
    mae = mean_absolute_error(all_targets_inv, all_preds_inv)
    r2= r2_score(all_targets_inv, all_preds_inv)
    rmse=mean_squared_error(all_targets_inv, all_preds_inv, squared=False)
    print('r2=',r2)
    print('rmse=',rmse)
    print('mae=',mae)

In [ ]:
for uap_loaded in uap_list005:
    original_preds, adversarial_preds, targets, original_weathers, adversarial_weathers = adversarial_attack_inde(uap_loaded, model, test_loader, device=device)
    adversarial_preds_uni = np.concatenate(adversarial_preds).flatten()
    targets= np.concatenate(targets).flatten()
    all_targets_inv=wind_power_scaler.inverse_transform(np.array(targets).reshape(-1, 1)).squeeze()
    all_preds_inv=wind_power_scaler.inverse_transform(np.array(adversarial_preds_uni).reshape(-1, 1)).squeeze()
    mae = mean_absolute_error(all_targets_inv, all_preds_inv)
    r2= r2_score(all_targets_inv, all_preds_inv)
    rmse=mean_squared_error(all_targets_inv, all_preds_inv, squared=False)
    print('r2=',r2)
    print('rmse=',rmse)
    print('mae=',mae)

In [ ]:
for uap_loaded in uap_list003:
    original_preds, adversarial_preds, targets, original_weathers, adversarial_weathers = adversarial_attack_inde(uap_loaded, model, test_loader, device=device)
    adversarial_preds_uni = np.concatenate(adversarial_preds).flatten()
    targets= np.concatenate(targets).flatten()
    all_targets_inv=wind_power_scaler.inverse_transform(np.array(targets).reshape(-1, 1)).squeeze()
    all_preds_inv=wind_power_scaler.inverse_transform(np.array(adversarial_preds_uni).reshape(-1, 1)).squeeze()
    mae = mean_absolute_error(all_targets_inv, all_preds_inv)
    r2= r2_score(all_targets_inv, all_preds_inv)
    rmse=mean_squared_error(all_targets_inv, all_preds_inv, squared=False)
    print('r2=',r2)
    print('rmse=',rmse)
    print('mae=',mae)

## RUP

In [ ]:
with open('rup_ensemble_003_caiso.pkl', 'rb') as f:
    rup_ensemble_003 = pickle.load(f)

with open('rup_ensemble_005_caiso.pkl', 'rb') as f:
    rup_ensemble_005 = pickle.load(f)

with open('rup_ensemble_010_caiso.pkl', 'rb') as f:
    rup_ensemble_010 = pickle.load(f)

rup_list=[rup_ensemble_003,rup_ensemble_005,rup_ensemble_010]

In [ ]:
for uap_loaded in rup_list:
    original_preds, adversarial_preds, targets, original_weathers, adversarial_weathers = adversarial_attack_inde(uap_loaded, model, test_loader, device=device)
    adversarial_preds_uni = np.concatenate(adversarial_preds).flatten()
    targets= np.concatenate(targets).flatten()
    all_targets_inv=wind_power_scaler.inverse_transform(np.array(targets).reshape(-1, 1)).squeeze()
    all_preds_inv=wind_power_scaler.inverse_transform(np.array(adversarial_preds_uni).reshape(-1, 1)).squeeze()
    mae = mean_absolute_error(all_targets_inv, all_preds_inv)
    r2= r2_score(all_targets_inv, all_preds_inv)
    rmse=mean_squared_error(all_targets_inv, all_preds_inv, squared=False)
    print('r2=',r2)
    print('rmse=',rmse)
    print('mae=',mae)

## RUPW

In [ ]:
with open('RUPW010_caiso.pkl', 'rb') as f:
    rupw_010 = pickle.load(f)

with open('RUPW005_caiso.pkl', 'rb') as f:
    rupw_005 = pickle.load(f)

with open('RUPW003_caiso.pkl', 'rb') as f:
    rupw_003 = pickle.load(f)

rupw_ensemble=[rupw_003,rupw_005,rupw_010]

In [ ]:
for uap_loaded in rupw_ensemble:
    original_preds, adversarial_preds, targets, original_weathers, adversarial_weathers = adversarial_attack_inde(uap_loaded, model, test_loader, device=device)
    adversarial_preds_uni = np.concatenate(adversarial_preds).flatten()
    targets= np.concatenate(targets).flatten()
    all_targets_inv=wind_power_scaler.inverse_transform(np.array(targets).reshape(-1, 1)).squeeze()
    all_preds_inv=wind_power_scaler.inverse_transform(np.array(adversarial_preds_uni).reshape(-1, 1)).squeeze()
    mae = mean_absolute_error(all_targets_inv, all_preds_inv)
    r2= r2_score(all_targets_inv, all_preds_inv)
    rmse=mean_squared_error(all_targets_inv, all_preds_inv, squared=False)
    print('r2=',r2)
    print('rmse=',rmse)
    print('mae=',mae)